<a href="https://colab.research.google.com/github/aman190202/PointNet/blob/main/PointCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random
import math
!pip install path
from path import Path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

path = Path("ModelNet10")

--2022-11-28 16:53:42--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.74
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.74|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip [following]
--2022-11-28 16:53:43--  https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip’

ModelNet10.zip      100%[===================>] 451.47M  12.8MB/s    in 37s     

2022-11-28 16:54:21 (12.3 MB/s) - ‘ModelNet10.zip’ saved [473402300/473402300]



In [4]:
def read_off(file):
  if 'OFF' !=file.readline().strip():
    raise('Not a valid OFF header')
  n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
  verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
  faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
  return verts, faces

with open(path/"bed/train/bed_0003.off",'r') as f:
  mesh = read_off(f)

In [5]:
import plotly.graph_objects as go

In [6]:
x_data = []
y_data = []
z_data = []
for i in mesh[0]:
  x_data.append(i[0])
  y_data.append(i[1])
  z_data.append(i[2])

fig = go.Figure(data=[go.Mesh3d( 
  x=x_data, y=y_data, z=z_data, color='blue', opacity=0.70)]) 
fig.show()

In [7]:
verts, faces = mesh
areas = np.zeros((len(faces)))
verts = np.array(verts)

# function to calculate triangle area by its vertices
# https://en.wikipedia.org/wiki/Heron%27s_formula
def triangle_area(pt1, pt2, pt3):
    side_a = np.linalg.norm(pt1 - pt2)
    side_b = np.linalg.norm(pt2 - pt3)
    side_c = np.linalg.norm(pt3 - pt1)
    s = 0.5 * ( side_a + side_b + side_c)
    return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

# we calculate areas of all faces in our mesh
for i in range(len(areas)):
    areas[i] = (triangle_area(verts[faces[i][0]],
                              verts[faces[i][1]],
                              verts[faces[i][2]]))

In [8]:
k = 3000
# we sample 'k' faces with probabilities proportional to their areas
# weights are used to create a distribution.
# they don't have to sum up to one.
sampled_faces = (random.choices(faces, 
                                weights=areas,
                                k=k))

# function to sample points on a triangle surface
def sample_point(pt1, pt2, pt3):
    # barycentric coordinates on a triangle
    # https://mathworld.wolfram.com/BarycentricCoordinates.html
    s, t = sorted([random.random(), random.random()])
    f = lambda i: s * pt1[i] + (t-s) * pt2[i] + (1-t) * pt3[i]
    return (f(0), f(1), f(2))
 
pointcloud = np.zeros((k, 3))

# sample points on chosen faces for the point cloud of size 'k'
for i in range(len(sampled_faces)):
    pointcloud[i] = (sample_point(verts[sampled_faces[i][0]],
                                  verts[sampled_faces[i][1]],
                                  verts[sampled_faces[i][2]]))

In [10]:
pointcloud

(3000, 3)

In [11]:
!pip install ipyvolume
import ipyvolume as ipv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 28.7 MB/s 
     |████████████████████████████████| 3.3 MB 53.9 MB/s 
     |████████████████████████████████| 260 kB 69.7 MB/s 
     |████████████████████████████████| 1.6 MB 62.4 MB/s 
     |████████████████████████████████| 271 kB 68.8 MB/s 


TypeError: ignored